In [51]:
import requests
from bs4 import BeautifulSoup

def scrape(
        url: str,
        number_headlines: int,
        printi: bool) -> list:
    """
    Scrape data from Google News
    """
    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, 'html.parser')
    headlines = soup.find_all('a', class_='JtKRv')
    all_headlines = [i.text for i in headlines]

    if printi:
        print(all_headlines)

    return all_headlines[:number_headlines]

scrape_data = scrape(
    url="https://news.google.com/search?for=shiba+inu+coin&hl=en-US&gl=US&ceid=US%3Aen",
    number_headlines=30,
    printi=True
)

['Shiba Inu price could rise 60% as burn rate rises 3,400%', '33.7 Trillion Shiba Inu (SHIB) Incoming in Next 24 Hours', 'Here’s Why Shiba Inu Price Could Double Soon', 'Shiba Inu Price Prediction: Can SHIB Surge 1000% in Bull Run As Next-Gen Altcoins Skyrocket', 'Shiba Inu Coin Is Set For Major Breakouts, Here Are The Next Two Altcoins To Follow The Same Trend |', 'Dogecoin Price Climbs as Shiba Inu Spikes to 8-Month High', 'Shiba Inu Price Prediction: SHIB To Reach $0.004 ATH Before January As DOGE Rival Eyes 14,000% Rally In 1 Week', 'Shiba Inu Soars To Eight-Month High, Up 13% as Meme Coins Rally', 'The 3 Best Shiba Inu (SHIB) Substitutes That Might Make $700000 from $700 in the Next 7 Months', "$2 and $110 Billion XRP: What's Next? Shiba Inu (SHIB) Enormous Move for Erasing Zero, Pepe (PEPE) Fundamental Breakout Secured", 'Shiba Inu Coin Alternative Is Expected To Reach $1 Before SHIB: Priced At $0.012, This Viral Altcoin Has', 'Shiba Inu Price Prediction: SHIB Burn Rate Dips, Wil

In [52]:
import yaml

with open('src/modules/news_summary/tickers.yaml', 'r') as file:
    file = yaml.safe_load(file)

In [59]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv('src/.env')


llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-turbo")

def summarizer(
        prompt_path: str,
        llm: ChatOpenAI,
        headlines: list,
    ):
    "Summarize Google News data"
    with open(prompt_path, 'r', encoding='utf-8') as file:
        markdown_string = file.read()
    prompt_template = PromptTemplate(template=markdown_string, input_variables=["headlines"])
    chain = prompt_template | llm | StrOutputParser()


    return chain.invoke({"headlines": headlines})


result = summarizer(prompt_path="src/prompts/news_summarizer.md", llm=llm, headlines=scrape_data)
result

KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "Key_Insights"\'}.  Expected: [\'\\n  "Key_Insights"\', \'headlines\'] Received: [\'headlines\']'

In [49]:
for key, value in file.items():

    scrape_data = scrape(
        url=value,
        number_headlines=30,
        printi=True
        )

    print(summarizer(
        prompt_path="src/prompts/news_analyst.md",
        llm=llm,
        headlines=scrape_data
        ))

['Top 3 Price Prediction Bitcoin, Ethereum, Ripple: BTC consolidates while ETH, XRP rallies', 'Ethereum (ETH) Price Prediction for November 30', 'Ethereum Price Forecast: ETH could see a new all-time high as SEC may approve staking in ETH ETFs', 'Ethereum co-founder moves $72 million in ETH to Kraken', "Ethereum 'dying a slow death' as ETH breaks 8-year trend vs. Bitcoin", 'Ethereum inflation soars amid Dencun changes—less than 100k ETH away from pre-Merge levels', 'ETH 3.0: Sharding may return to scale Ethereum to millions of TPS', 'Ethereum Price Prediction: Analysts Forecast $10,000 ETH by 2025', 'Ethereum Price Today: ETH Hints at $3,000 Revisit', 'Ethereum (ETH) Price: Shows Strength Above $3,600 Amid Whale Activity', 'Steno Research: Ethereum Transaction Revenue Skyrockets Post Trump’s Victory', 'Ethereum (ETH) on Track to Flip $4K – Ripple (XRP) Hits $2 but Traders Bet More on This New AI Crypto With 50x Upside Potential', 'Coinbase Refutes Accusations of Not Boosting Ethereum S

KeyboardInterrupt: 